In [1]:
!pip install transformers
!pip install torch

In [2]:
import pandas as pd
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
df_guj = pd.read_csv('/content/gdrive/MyDrive/HASOC23/guj_train_translated_combined.csv', delimiter = ',')
df_guj

,tweet_id,created_at,text,user_screen_name,label,translated,combined
0,guj001,2023-05-19 15:44:55+00:00,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...,Red__chilli_,HOF,@username @username e waydi na thou hes dobo\n...,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...
1,guj002,2023-05-27 16:44:40+00:00,@username @username આવા હલકા કેટલા છે કાય ખબર ...,sureshn25559831,HOF,@username @username does not know how much is ...,@username @username આવા હલકા કેટલા છે કાય ખબર ...
2,guj003,2023-05-25 03:56:05+00:00,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...,chattansinh,HOF,@username @username like you are not in the ca...,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...
3,guj004,2023-03-25 08:05:22+00:00,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...,imkpbhatt,NOT,@username is also a fucking people.\n\n@userna...,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...
4,guj005,2023-05-24 07:01:42+00:00,"@username @username બેશક, હું તમને મૂર્ખ કહી શ...",irshadvihari,NOT,"@username @username Of course, I can call you ...","@username @username બેશક, હું તમને મૂર્ખ કહી શ..."
...,...,...,...,...,...,...,...
195,guj196,2023-03-25 16:10:38+00:00,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...,nitinchavda548,NOT,@username has written a lot of Rahul Gandhi in...,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...
196,guj197,2023-05-26 12:27:01+00:00,@username @username @username ભડવો નયન ચાદર મો...,chattansinh,HOF,@username @username @username,@username @username @username ભડવો નયન ચાદર મો...
197,guj198,2023-05-27 07:26:02+00:00,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...,Rana_Naidu_,HOF,@username Chodina Tara and Pakistan Gathering ...,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...
198,guj199,2023-03-26 09:32:06+00:00,@username @username @username @username @usern...,kkpatel852,NOT,@username @username @username @username @usern...,@username @username @username @username @usern...


In [4]:
import torch
from transformers import BertModel, BertTokenizerFast


tokenizer = BertTokenizerFast.from_pretrained("sentence-transformers/LaBSE")
if tokenizer.pad_token is None:
  print('adding token')
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})
bertmodel = BertModel.from_pretrained("sentence-transformers/LaBSE")
bertmodel.resize_token_embeddings(len(tokenizer))
bertmodel.eval()

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embeding dimension will be 501153. This might induce some performance reduction as *Tensor Cores* will not be available. For more details  about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(501153, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
 

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df_guj['label'] = le.fit_transform(df_guj.label.values)
df_guj

,tweet_id,created_at,text,user_screen_name,label,translated,combined
0,guj001,2023-05-19 15:44:55+00:00,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...,Red__chilli_,0,@username @username e waydi na thou hes dobo\n...,@username @username ઇ વાયડાઈ ના તું હાઈસ ડોબો ...
1,guj002,2023-05-27 16:44:40+00:00,@username @username આવા હલકા કેટલા છે કાય ખબર ...,sureshn25559831,0,@username @username does not know how much is ...,@username @username આવા હલકા કેટલા છે કાય ખબર ...
2,guj003,2023-05-25 03:56:05+00:00,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...,chattansinh,0,@username @username like you are not in the ca...,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...
3,guj004,2023-03-25 08:05:22+00:00,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...,imkpbhatt,1,@username is also a fucking people.\n\n@userna...,@username આવું વાહિયાત ખાવા વાળા લોકો પણ છે 😈😈...
4,guj005,2023-05-24 07:01:42+00:00,"@username @username બેશક, હું તમને મૂર્ખ કહી શ...",irshadvihari,1,"@username @username Of course, I can call you ...","@username @username બેશક, હું તમને મૂર્ખ કહી શ..."
...,...,...,...,...,...,...,...
195,guj196,2023-03-25 16:10:38+00:00,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...,nitinchavda548,1,@username has written a lot of Rahul Gandhi in...,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...
196,guj197,2023-05-26 12:27:01+00:00,@username @username @username ભડવો નયન ચાદર મો...,chattansinh,0,@username @username @username,@username @username @username ભડવો નયન ચાદર મો...
197,guj198,2023-05-27 07:26:02+00:00,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...,Rana_Naidu_,0,@username Chodina Tara and Pakistan Gathering ...,@username ચોદીના તારા જેવા ને પાકિસ્તાન ભેગી ન...
198,guj199,2023-03-26 09:32:06+00:00,@username @username @username @username @usern...,kkpatel852,1,@username @username @username @username @usern...,@username @username @username @username @usern...


In [6]:
#labels = torch.FloatTensor([1,0,0,1,0,1,0,1])


In [7]:
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
class sinhalese_data(Dataset):
  def __init__(self, df_sinh, tokenizer):
    self.dataset = df_sinh
    self.data = list(self.dataset.combined)
    self.labels = list(self.dataset.label)
    self.tokenizer = tokenizer
    #model = BertModel.from_pretrained("setu4993/LaBSE")

  def __len__(self):
    return len(self.data)
  def __getitem__(self, index):
    example = self.data[index]
    inputs = tokenizer(example, return_tensors="pt", padding='max_length', max_length=512, truncation=True)
    input_ids = inputs['input_ids'].squeeze()
    attention_mask = inputs['attention_mask'].squeeze()
    #print(self.labels[index])
    labels = self.labels[index]
    return input_ids, attention_mask, labels

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [9]:
from transformers import BertModel
from torch import nn
class CustomBERTModel(nn.Module):
    def __init__(self, model):
          super(CustomBERTModel, self).__init__()
          self.bert = model
          ### New layers:

          self.linear1 = nn.Linear(768, 512)
          self.dropout = nn.Dropout(0.3)
          self.relu = nn.LeakyReLU(negative_slope=0.01)
          self.linear2 = nn.Linear(512, 256)


          self.linear3 = nn.Linear(256, 128)


          self.linear4 = nn.Linear(128, 1) ## 2 classes in this example
          self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
          output = self.bert(
               input_ids,
               attention_mask=attention_mask)
          #print(output)
          pooler_output = output['pooler_output']
          #print(sequence_output.shape())
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(pooler_output)
          relu1_output = self.relu(linear1_output)
          dropout1_output = self.dropout(relu1_output)

          linear2_output = self.linear2(dropout1_output)
          relu2_output = self.relu(linear2_output)


          linear3_output = self.linear3(relu2_output)
          relu3_output = self.relu(linear3_output)

          #sigmoid_output = self.sigmoid(linear2_output)

          linear4_output = self.linear4(relu3_output)
          sigmoid_output = self.sigmoid(linear4_output)
          return sigmoid_output

In [10]:
from transformers import BertModel
from torch import nn
from huggingface_hub import PyTorchModelHubMixin
class CustomBERTModel(nn.Module,PyTorchModelHubMixin):
    def __init__(self, model):
          super().__init__()
          self.bert = model
          ### New layers:
          self.dropout = nn.Dropout(0.3)
          self.linear1 = nn.Linear(768, 256)
          self.relu = nn.LeakyReLU(negative_slope=0.01)
          self.linear2 = nn.Linear(256, 1) ## 2 is the number of classes in this example
          self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask):
          output = self.bert(
               input_ids,
               attention_mask=attention_mask)
          #print(output)
          pooler_output = self.dropout(output['pooler_output'])
          #print(sequence_output.shape())
          # sequence_output has the following shape: (batch_size, sequence_length, 768)
          linear1_output = self.linear1(pooler_output) ## extract the 1st token's embeddings
          relu_output = self.relu(linear1_output)
          linear2_output = self.linear2(relu_output)
          sigmoid_output = self.sigmoid(linear2_output)

          return sigmoid_output

In [11]:
model = CustomBERTModel(bertmodel)
model = model.eval()
model.to(device)

CustomBERTModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_guj, test_size=0.15, stratify = df_guj['label'], random_state=42)

In [13]:
df_train

,tweet_id,created_at,text,user_screen_name,label,translated,combined
193,guj194,2023-05-27 05:31:34+00:00,‘કુસ્તી સંગઠનના વડા બહેન-દીકરીઓ વિશે વાહિયાત વ...,GSTV_NEWS,1,'The head of the wrestling organization talks ...,‘કુસ્તી સંગઠનના વડા બહેન-દીકરીઓ વિશે વાહિયાત વ...
55,guj056,2023-05-28 05:37:08+00:00,@username સૂરજ ભડવો ભો####,veer_kathi,0,@username sun,@username સૂરજ ભડવો ભો#### @username sun
9,guj010,2023-05-22 11:17:23+00:00,"@username આ બધા કામ લુખ્ખા પાર્ટી જ કરે,,એટલે ...",BeingSohebpatel,0,@username does all this work on the lucky part...,"@username આ બધા કામ લુખ્ખા પાર્ટી જ કરે,,એટલે ..."
25,guj026,2023-03-26 06:56:30+00:00,"ગુજરાત પોલીસ સ્ટેડિયમમાં ચાના કપ પર લખેલું છે,...",Chandanjit11,0,The Gujarat Police Stadium is written on a cup...,"ગુજરાત પોલીસ સ્ટેડિયમમાં ચાના કપ પર લખેલું છે,..."
129,guj130,2023-05-27 06:33:15+00:00,@username @username આવા ને દલાલ કહેવાય સત્તા ન...,DhavalP67910501,0,@username @username such as a broker,@username @username આવા ને દલાલ કહેવાય સત્તા ન...
...,...,...,...,...,...,...,...
31,guj032,2023-05-24 10:04:59+00:00,@username @username @username ગુજરાત માં લોકો ...,Jitendra1682,0,@username @username @username is falling in Gu...,@username @username @username ગુજરાત માં લોકો ...
92,guj093,2023-05-24 11:51:07+00:00,😂 બે ચોર એક દુકાન માં ચોરી\nકરવા ગયાં \n\nશર્ટ...,ChakaliChoro,1,ર two thieves stolen in a shop\nGone\n\nPrice ...,😂 બે ચોર એક દુકાન માં ચોરી\nકરવા ગયાં \n\nશર્ટ...
160,guj161,2023-05-25 11:48:54+00:00,જામનગરમાં જામ્યો પંખીઓનો કલરવ... 🦚🕊️\n\n📍લખોટા...,hakubhajamnagar,1,The tweet of the birds frozen in Jamnagar ... ...,જામનગરમાં જામ્યો પંખીઓનો કલરવ... 🦚🕊️\n\n📍લખોટા...
140,guj141,2023-05-27 11:08:15+00:00,@username વાહિયાત વિવાદ છે...અને મિડિયા તેમાં ...,JaydeepsinhJv,1,@username is a absurd dispute ... and the medi...,@username વાહિયાત વિવાદ છે...અને મિડિયા તેમાં ...


In [14]:
dataset = sinhalese_data(df_train, tokenizer)
valset = sinhalese_data(df_test, tokenizer)

In [15]:
import torch
from torch.nn import functional as F

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.BCELoss()
batch_size = 4
num_epochs = 7
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size = batch_size, shuffle=True)

In [16]:
import numpy as np
from torch import nn
import torch.nn.functional as F
criterion = nn.BCELoss()
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    val_loss = 0
    print('Epoch: ', epoch)

    for batch in data_loader:
        # Move batch tensors to the device
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        labels = labels.reshape(len(labels),1)
        labels = labels.float()
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask = attention_mask)
               #print(outputs)
        #outputs = F.sigmoid(outputs)
       #print(outputs.shape, labels.shape)
        #print('Output: ', outputs, '\nLabels: ', labels)
        loss = criterion(outputs, labels.reshape(len(labels),1))
        #print(loss)
        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    if (epoch%5==0) and epoch!=0:
     for batch in val_loader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        labels = labels.reshape(len(labels),1)
        labels = labels.float()
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids=input_ids,attention_mask = attention_mask)
        #print(outputs)
        #outputs = F.sigmoid(outputs)
        #print(outputs.shape, labels.shape)
        #print('Output: ', outputs, '\nLabels: ', labels)
        loss = criterion(outputs, labels.reshape(len(labels),1))
        val_loss+=loss.item()


    avg_loss = total_loss / (len(data_loader)*batch_size)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {avg_loss:.4f} - Validation Loss: {val_loss/(len(val_loader)*batch_size)}")

Epoch:  0
Epoch 1/7 - Average Loss: 0.1653 - Validation Loss: 0.0
Epoch:  1
Epoch 2/7 - Average Loss: 0.1333 - Validation Loss: 0.0
Epoch:  2
Epoch 3/7 - Average Loss: 0.0912 - Validation Loss: 0.0
Epoch:  3
Epoch 4/7 - Average Loss: 0.0458 - Validation Loss: 0.0
Epoch:  4
Epoch 5/7 - Average Loss: 0.0245 - Validation Loss: 0.0
Epoch:  5
Epoch 6/7 - Average Loss: 0.0395 - Validation Loss: 0.15827196347527206
Epoch:  6
Epoch 7/7 - Average Loss: 0.0073 - Validation Loss: 0.0


Translated tweet - BERT/or any other

VS

Low resource language - LaBSE

VS

Ensemble based on voting.

VS

LaBSE - with low resource + english translation. (Maybe something missed out in embeddings).

In [17]:
df_test

,tweet_id,created_at,text,user_screen_name,label,translated,combined
164,guj165,2023-05-24 15:35:30+00:00,તેલના ડબ્બા નાં ભાવ ઘટવાનું શરૂ થયુ છે\n\nકોંગ...,RWGujarat,0,The price of oil bins has begun to fall\n\nIf ...,તેલના ડબ્બા નાં ભાવ ઘટવાનું શરૂ થયુ છે\n\nકોંગ...
71,guj072,2023-05-26 08:52:15+00:00,@username @username Gujarat નું અપમાન તો કોંગ્...,raj869180,1,@username @username Gujarat insults Congress e...,@username @username Gujarat નું અપમાન તો કોંગ્...
2,guj003,2023-05-25 03:56:05+00:00,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...,chattansinh,0,@username @username like you are not in the ca...,@username @username તારી જેમ અમે જ્ઞાતિ માં બટ...
72,guj073,2023-05-26 09:45:01+00:00,@username @username @username 9 વર્ષ મા ઉદ્યોગ...,AshokBabariya8,1,@username @username @username businessmen in 9...,@username @username @username 9 વર્ષ મા ઉદ્યોગ...
14,guj015,2023-05-24 07:03:09+00:00,ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર તાલુકામ...,PinalPa10542589,1,In Jambusar taluka in Bharuch district in Guja...,ગુજરાત માં ભરૂચ જિલ્લામાં આવેલો જંબુસર તાલુકામ...
148,guj149,2023-05-19 05:20:55+00:00,@username વિરાટ હિન્દુ ને ગઈ રાત નોં ઉજાગરો સે...,Are_O_Sambha,0,@username Virat Hindu to light last night ...,@username વિરાટ હિન્દુ ને ગઈ રાત નોં ઉજાગરો સે...
83,guj084,2023-05-28 06:30:00+00:00,પારકી પંચાત' - જુઓ સોમ થી શનિ બપોરે 3 વાગ્યે. ...,Divya_Bhaskar,1,Parki Panchayat ' - See Mon to Saturday at 3pm...,પારકી પંચાત' - જુઓ સોમ થી શનિ બપોરે 3 વાગ્યે. ...
12,guj013,2023-05-19 06:46:15+00:00,@username @username આ હું તો બોલીશ વાળા રોનક પ...,Amru1967,1,@username @username.\nThe program is not compl...,@username @username આ હું તો બોલીશ વાળા રોનક પ...
69,guj070,2023-05-21 06:33:53+00:00,જુઢો એ કાળિયો કહે મુજ વગર એને કોઈ ના ફાવે.\nગો...,Mutradifalfaz,1,The knife does not say no one without me.\nHow...,જુઢો એ કાળિયો કહે મુજ વગર એને કોઈ ના ફાવે.\nગો...
195,guj196,2023-03-25 16:10:38+00:00,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...,nitinchavda548,1,@username has written a lot of Rahul Gandhi in...,@username આજ વિદેશી પ્રિન્ટ મીડિયા માં રાહુલ ગ...


In [18]:
torch.save(model, '/content/gdrive/MyDrive/saved_models/last-LA-BSE-Sigmoid-deep-translated-7ep-guj')
